#### This notebook takes care of pulling the raw data from the Alpha Vantage API and writing it to csv files.

#### 1) Orginal plan was to pull all available daily and hourly data for the two big index ETFs, SPY and QQQ
####    However, no luck with the hourly data. I thought technical data went back more than the first "slice" but no luck.
####    So for now, I'm going to expand on the number of ETFs for growing the dataset. In the future, I could calculate the
####    technical indicators for hourly data (or find a source that likely isn't free).

In [3]:
import requests
import pandas as pd
import time

In [5]:
# symbols and technical indicators [code, interval, name]
# https://www.alphavantage.co/documentation/#technical-indicators
#
# got rid of JNK (weirdly high open z-score mean), HYG (weirdly low low z-score mean), and EWZ/IEF (infinite end values)

#symbol_list = ['SPY','QQQ','XLF','EEM','XLE','SLV','FXI','GDX','EFA','TLT','LQD','XLU','XLV','XLI','IEMG','VWO','XLK','IEF','XLB','JETS','BND']
symbol_list = ['SPY']
tech_list = [['SMA',50,'Technical Analysis: SMA'],
             ['EMA',21,'Technical Analysis: EMA'],
             ['RSI',14,'Technical Analysis: RSI']]

for symbol in symbol_list:

    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&apikey=PDS8Y8E8KULJVDET"
    r = requests.get(url)
    data = r.json()
    df_price = pd.DataFrame(data['Time Series (Daily)']).T
    print(df_price.head())
    time.sleep(15)

    for tech in tech_list:
        url = f"https://www.alphavantage.co/query?function={tech[0]}&symbol={symbol}&interval=daily&time_period={tech[1]}&series_type=close&apikey=PDS8Y8E8KULJVDET"
        r = requests.get(url)
        data = r.json()
        df_tech = pd.DataFrame(data[tech[2]]).T
        df_price = df_price.merge(df_tech, how='inner', left_index=True, right_index=True)
        time.sleep(15)
    
    df_price.to_csv(f"../data/raw/{symbol}_daily.csv")
    print(f"{symbol} saved")


           1. open 2. high    3. low 4. close 5. adjusted close  6. volume  \
2021-12-07  464.41  468.88  458.6546   468.28            468.28   92791114   
2021-12-06  456.13  460.79    453.56   458.79            458.79   98977532   
2021-12-03  459.17   460.3    448.92   453.42            453.42  137331647   
2021-12-02  450.73  459.07    450.31    457.4             457.4  127637758   
2021-12-01  461.64  464.67    450.29    450.5             450.5  132485835   

           7. dividend amount 8. split coefficient  
2021-12-07             0.0000                  1.0  
2021-12-06             0.0000                  1.0  
2021-12-03             0.0000                  1.0  
2021-12-02             0.0000                  1.0  
2021-12-01             0.0000                  1.0  
SPY saved
